Installing req packages

In [3]:
!pip install transformers
!pip install git+https://github.com/openai/CLIP.git
!pip install ftfy regex tqdm
!pip install torchvision
!pip install pillow

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-h9iw36bc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-h9iw36bc
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=f2541ae957c38a2003ef8c7b59440a003863047ebb4110d57b3deaf025079798
  Stored in directory: /tmp/pip-ephem-wheel-cache-axvmral9/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


# **Image Processing & Issue ranking**

## **Phase 1**
Takes image as input --> Subset processing --> Caption generation --> Caption ranking --> Knowledge base comparison --> Ranked output


In [ ]:
# Required Imports
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import clip
import torchvision.transforms as transforms
import numpy as np

# Load models globally
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load BLIP model
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
blip_model.eval()

# Load CLIP model
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

# Load the issue-specific knowledge base
def load_knowledge_base(path="issue_knowledge.txt"):
    with open(path, "r") as f:
        return [line.strip() for line in f.readlines() if line.strip()]

# Main function to detect issues in the image
def detect_image_issues(image_path, knowledge_base_path="issue_knowledge.txt", top_k=3):
    image = Image.open(image_path).convert('RGB')

    # Step 1: Generate candidate captions using BLIP
    inputs = blip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        caption_output = blip_model.generate(
            **inputs,
            max_length=100,
            num_beams=5,
            num_return_sequences=5,
            early_stopping=True
        )
    captions = [blip_processor.decode(out, skip_special_tokens=True) for out in caption_output]

    # Step 2: Load issue-specific prompts
    issue_prompts = load_knowledge_base(knowledge_base_path)
    text_inputs = torch.cat([clip.tokenize(prompt) for prompt in issue_prompts]).to(device)

    # Step 3: Process image with CLIP
    image_input_clip = clip_preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = clip_model.encode_image(image_input_clip)
        text_features = clip_model.encode_text(text_inputs)

    # Step 4: Compute cosine similarity between image and KB issues
    similarity_scores = (image_features @ text_features.T).squeeze(0)
    scores_np = similarity_scores.cpu().numpy()

    # Step 5: Robust filtering
    mean_score = scores_np.mean()
    std_score = scores_np.std()
    max_score = scores_np.max()
    dynamic_threshold = mean_score + std_score
    hard_min_cutoff = 0.3
    selected_threshold = max(dynamic_threshold, hard_min_cutoff)

    # If top score is below the safe threshold, fallback to BLIP captions
    if max_score < selected_threshold:
        print("\n[Fallback Mode Triggered]")
        print("No strong matches in knowledge base. Showing raw issue predictions:")
        for idx, caption in enumerate(captions[:top_k], 1):
            print(f"{idx}. {caption}")
        return captions[:top_k]

    # Step 6: Ensure top issue is significantly above average (safety margin)
    top_indices = similarity_scores.topk(top_k).indices
    top_issues = [(issue_prompts[i], float(similarity_scores[i])) for i in top_indices]

    top_score = float(similarity_scores[top_indices[0]])
    second_score = float(similarity_scores[top_indices[1]]) if top_k > 1 else 0.0
    if top_score - second_score < 0.07:  # Not a strong enough margin
        print("\n[Uncertain Prediction Warning]")
        print("Predictions too close. Falling back to general BLIP captions:")
        for idx, caption in enumerate(captions[:top_k], 1):
            print(f"{idx}. {caption}")
        return captions[:top_k]

    # Output top K issues
    print("\nTop Detected Issues (from knowledge base):")
    for idx, (issue, score) in enumerate(top_issues, 1):
        print(f"{idx}. {issue} (Score: {score:.4f})")

    return [issue for issue, _ in top_issues]

In [ ]:
image_path = "/content/water floor.jpeg"
kb_path = "/content/issue_knowledge.txt"

issues = detect_image_issues(image_path, knowledge_base_path=kb_path, top_k=3)


Top Detected Issues (from knowledge base):
1. Floor tilting toward one corner (Score: 35.5445)
2. Improper waterproofing layer (Score: 34.1083)
3. Detached skirting on cement floor (Score: 31.8246)


# **Agent 1 with added features**

In [4]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=8fd0e8099cf23a8dfb9a6fe43fadc6fb48bcef05cdabe265e2005a04c8dec828
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


## **Contractor contact code**
Google search

In [ ]:
from serpapi import GoogleSearch

def find_contractor_websites(issue_type: str, location: str, num_results: int = 3, api_key: str = "YOUR_SERPAPI_KEY"):
    query = f"{issue_type} repair contractor near {location}"

    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key,
        "num": num_results
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # Print the full results to understand the returned data structure
    print("\n🌐 Search Results:", results)

    contractor_links = []

    # Check 'local_results' for contractor info if available
    if "local_results" in results:
        for contractor in results["local_results"][:num_results]:
            link = contractor.get("link")
            name = contractor.get("title", "Unknown")
            if link:
                contractor_links.append({"name": name, "link": link})

    # Fallback to 'organic_results' if 'local_results' are not found
    if not contractor_links and "organic_results" in results:
        for result in results["organic_results"][:num_results]:
            title = result.get("title", "Unknown")
            link = result.get("link")
            if link:
                contractor_links.append({"name": title, "link": link})

    # If no contractor results found, check 'related_searches' for helpful links
    if not contractor_links and "related_searches" in results:
        for related_search in results["related_searches"]:
            link = related_search.get("link")
            if link:
                contractor_links.append({"name": "Related Search", "link": link})

    # Return the contractor links found
    return contractor_links


### **Dry run**

In [ ]:
contractors = find_contractor_websites("mold removal", "Delhi", api_key="345a9c709f5ac9c5828a76c34c4611d3b25f11f3414a43ba54097349ced2f85e")
print("\nContractor Results:\n")
for idx, contractor in enumerate(contractors, 1):
    print(f"{idx}. {contractor['name']}")
    print(f"   🔗 {contractor['link']}")



🌐 Search Results: {'search_metadata': {'id': '6804ea14e4ae2e22f5344e46', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/5e21d87557f8f28e/6804ea14e4ae2e22f5344e46.json', 'created_at': '2025-04-20 12:35:32 UTC', 'processed_at': '2025-04-20 12:35:32 UTC', 'google_url': 'https://www.google.com/search?q=mold+removal+repair+contractor+near+Delhi&oq=mold+removal+repair+contractor+near+Delhi&num=3&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/5e21d87557f8f28e/6804ea14e4ae2e22f5344e46.html', 'total_time_taken': 7.29}, 'search_parameters': {'engine': 'google', 'q': 'mold removal repair contractor near Delhi', 'google_domain': 'google.com', 'num': '3', 'device': 'desktop'}, 'search_information': {'query_displayed': 'mold removal repair contractor near Delhi', 'organic_results_state': 'Fully empty'}, 'related_searches': [{'block_position': 1, 'link': 'https://www.google.com/search?num=3&sca_esv=fd82287142e91de9&q=Residential+mold+removal+repair+con

## **Agent 1 integraded**

In [ ]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import clip
import torchvision.transforms as transforms
import requests
import json

# ---------------------- Model Setup ----------------------

device = "cuda" if torch.cuda.is_available() else "cpu"

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
blip_model.eval()

clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

# ---------------------- Constants ----------------------

SERPAPI_KEY = "345a9c709f5ac9c5828a76c34c4611d3b25f11f3414a43ba54097349ced2f85e"
GROQ_API_KEY = "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"
GROQ_MODEL = "llama3-8b-8192"

# ---------------------- Knowledge Base ----------------------

def load_knowledge_base(path="issue_knowledge.txt"):
    with open(path, "r") as f:
        return [line.strip() for line in f.readlines() if line.strip()]

# ---------------------- Image Processing ----------------------

def detect_image_issues(image_path, knowledge_base_path="issue_knowledge.txt", top_k=3):
    image = Image.open(image_path).convert('RGB')

    # Generate BLIP captions
    inputs = blip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        caption_output = blip_model.generate(
            **inputs, max_length=100, num_beams=5, num_return_sequences=5, early_stopping=True
        )
    captions = [blip_processor.decode(out, skip_special_tokens=True) for out in caption_output]

    # Load KB prompts
    issue_prompts = load_knowledge_base(knowledge_base_path)
    text_inputs = torch.cat([clip.tokenize(prompt) for prompt in issue_prompts]).to(device)

    # CLIP features
    image_input_clip = clip_preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = clip_model.encode_image(image_input_clip)
        text_features = clip_model.encode_text(text_inputs)

    # Similarity
    similarity_scores = (image_features @ text_features.T).squeeze(0)
    scores_np = similarity_scores.cpu().numpy()

    mean_score = scores_np.mean()
    std_score = scores_np.std()
    max_score = scores_np.max()
    threshold = max(mean_score + std_score, 0.3)

    if max_score < threshold:
        return [], captions[:top_k]  # Fallback mode

    top_indices = similarity_scores.topk(top_k).indices
    top_issues = [(issue_prompts[i], float(similarity_scores[i])) for i in top_indices]

    return [issue for issue, _ in top_issues], captions[:top_k]

# ---------------------- LLM Troubleshooter ----------------------

def generate_troubleshooting_advice(issues, captions, user_input=""):
    issues_text = issues if issues else ["None confidently identified."]

    prompt = "\n".join([
        "You are an expert home repair assistant.",
        "Below are details from image-based inspection and user description.",
        "\n[IDENTIFIED ISSUES]",
        *issues_text,
        "\n[IMAGE CAPTIONS]",
        *captions,
        f"\n[USER DESCRIPTION]\n{user_input or 'N/A'}",
        "\nBased on the above, suggest a detailed troubleshooting plan step by step, then suggest when to contact a contractor."
    ])

    try:
        response = requests.post(
            url="https://api.groq.com/openai/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {GROQ_API_KEY}",
                "Content-Type": "application/json"
            },
            json={
                "model": GROQ_MODEL,
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0.7
            }
        )

        data = response.json()
        if "choices" in data and data["choices"]:
            return data["choices"][0]["message"]["content"].strip()
        else:
            return "[LLM ERROR] No valid response from Groq API.\n\nResponse:\n" + json.dumps(data, indent=2)

    except Exception as e:
        return f"[EXCEPTION] While contacting Groq API: {str(e)}"

# ---------------------- Contractor Finder ----------------------

def find_local_contractors(query, location):
    params = {
        "engine": "google",
        "q": f"{query} near {location}",
        "api_key": SERPAPI_KEY,
        "num": "5"
    }

    try:
        response = requests.get("https://serpapi.com/search", params=params)
        data = response.json()

        contractors = []
        for result in data.get("organic_results", []):
            title = result.get("title")
            link = result.get("link")
            if title and link:
                contractors.append((title, link))

        return contractors
    except Exception as e:
        return f"[EXCEPTION] Contractor search failed: {str(e)}"

# ---------------------- Main Pipeline ----------------------

def process_and_troubleshoot(image_path, user_input=""):
    issues, captions = detect_image_issues(image_path)

    print("\n--- DETECTED OUTPUT ---")
    print("Top Issues:", issues)
    print("Top Captions:", captions)

    print("\n--- TROUBLESHOOTING SUGGESTIONS ---")
    advice = generate_troubleshooting_advice(issues, captions, user_input)
    print(advice)

    status = input("\nDid this solve your issue? (yes/no): ").strip().lower()
    if status == "no":
        location = input("Enter your location for contractor suggestions: ").strip()
        query = " ".join(issues or captions)
        contractors = find_local_contractors(query, location)

        print("\n--- NEARBY CONTRACTORS ---")
        if contractors:
            for name, url in contractors:
                print(f"🔧 {name} — {url}")
        else:
            print("No contractor links found. Try rephrasing or check your internet.")


In [ ]:
process_and_troubleshoot(
    image_path="/content/pipe damage.jpg",
    user_input="i dont know whats wrong here, im scared",
 )



--- DETECTED OUTPUT ---
Top Issues: ['Improperly embedded water pipe', 'Water trickling near roof joint', 'Leaking junction of water inlet pipe']
Top Captions: ['water leaking from a pipe in the ceiling', 'water leaking from a pipe in a basement', 'water leaking from a pipe in a house']

--- TROUBLESHOOTING SUGGESTIONS ---
I'm happy to help you troubleshoot the issues with your home's plumbing system!

**Step-by-Step Troubleshooting Plan:**

1. **Identify the locations of the leaks:** From the images and user description, it appears that there are three separate leaks:
	* Water leaking from a pipe in the ceiling
	* Water leaking from a pipe in the basement
	* Water leaking from a pipe in a house (not specified where exactly)
2. **Check the roof joint:** Since water is trickling near the roof joint, it's essential to inspect the joint for any signs of damage, wear, or improper installation. Look for any gaps, cracks, or mineral deposits that could be causing the leak.
3. **Inspect the 

## **Agent 1 ** beta

In [ ]:
import torch
from PIL import Image
import clip
import json
import requests
import torchvision.transforms as transforms
from transformers import BlipProcessor, BlipForConditionalGeneration

# ------------------- Model Setup -------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
blip_model.eval()

clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

# ------------------- Keys -------------------
SERPAPI_KEY = "345a9c709f5ac9c5828a76c34c4611d3b25f11f3414a43ba54097349ced2f85e"
GROQ_API_KEY = "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"
GROQ_MODEL = "llama3-8b-8192"

# ------------------- Utils -------------------
def save_top3_output(captions, issues, contractors, file_path="top3_results.json"):
    data = {
        "captions": captions,
        "issues": issues,
        "contractors": contractors
    }
    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)

def load_knowledge_base(path="/content/issue_knowledge.txt"):
    with open(path, "r") as f:
        return [line.strip() for line in f.readlines() if line.strip()]

# ------------------- Core Logic -------------------
def detect_image_issues(image_path, knowledge_base_path="issue_knowledge.txt", top_k=3):
    image = Image.open(image_path).convert('RGB')

    # BLIP captioning
    inputs = blip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        caption_output = blip_model.generate(**inputs, max_length=100, num_beams=5, num_return_sequences=5)
    captions = [blip_processor.decode(out, skip_special_tokens=True) for out in caption_output][:top_k]

    # CLIP similarity to fallback if captions aren't strong
    issue_prompts = load_knowledge_base(knowledge_base_path)
    image_input_clip = clip_preprocess(image).unsqueeze(0).to(device)
    text_inputs = torch.cat([clip.tokenize(p) for p in issue_prompts]).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image_input_clip)
        text_features = clip_model.encode_text(text_inputs)

    similarity_scores = (image_features @ text_features.T).squeeze(0)
    scores_np = similarity_scores.cpu().numpy()
    max_score = scores_np.max()
    threshold = max(scores_np.mean() + scores_np.std(), 0.3)

    if max_score < threshold:
        return [], captions  # Fallback: use only captions

    top_indices = similarity_scores.topk(top_k).indices
    top_issues = [(issue_prompts[i], float(similarity_scores[i])) for i in top_indices]
    return [i[0] for i in top_issues], captions

def generate_troubleshooting_advice(issues, captions, user_input="", sentiment="neutral", chat_history=None):
    chat_history = chat_history or []

    prompt_sections = [
        "You are a repair assistant. Provide accurate, no-fluff guidance.",
        f"\n[ISSUES IDENTIFIED]\n{', '.join(issues) if issues else 'None'}",
        f"\n[CAPTIONS]\n{'; '.join(captions)}",
        f"\n[USER MESSAGE]\n{user_input if user_input else 'None'}",
        f"\n[SENTIMENT]\n{sentiment}",
        f"\n[CHAT HISTORY]\n" + "\n".join(chat_history[-3:]) if chat_history else ""
    ]

    full_prompt = "\n".join(prompt_sections) + "\n\nRespond in under 2 paragraphs. Use bullet points if needed. No emojis."

    try:
        response = requests.post(
            url="https://api.groq.com/openai/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {GROQ_API_KEY}",
                "Content-Type": "application/json"
            },
            json={
                "model": GROQ_MODEL,
                "messages": [{"role": "user", "content": full_prompt}],
                "temperature": 0.5
            }
        )
        data = response.json()
        if "choices" in data and data["choices"]:
            return data["choices"][0]["message"]["content"].strip()
        return "No response from model."

    except Exception as e:
        return f"LLM call failed: {e}"

def find_local_contractors(query, location, max_results=3):
    params = {
        "engine": "google",
        "q": f"{query} technician contact near {location}",
        "api_key": SERPAPI_KEY,
        "num": "10"
    }

    try:
        response = requests.get("https://serpapi.com/search", params=params)
        data = response.json()
        contractors = []
        helpful_websites = []

        # Pull from local business listings first
        for r in data.get("local_results", []):
            name = r.get("title") or r.get("name")
            phone = r.get("phone")
            address = r.get("address")
            if name:
                contractors.append(f"{name} - {phone or 'N/A'} - {address or 'N/A'}")
            if len(contractors) >= max_results:
                break

        # Fallback: extract useful websites if fewer than required contractors found
        if len(contractors) < max_results:
            for org in data.get("organic_results", []):
                link = org.get("link")
                title = org.get("title")
                if link and title:
                    helpful_websites.append(f"{title} - {link}")
                if len(helpful_websites) >= max_results:
                    break

        results = []
        if contractors:
            results.append("Contractors:")
            results.extend(contractors)
        if helpful_websites:
            results.append("Websites that might help:")
            results.extend(helpful_websites)

        return results

    except Exception as e:
        return [f"Contractor search failed: {e}"]

# ------------------- Master Controller -------------------
def troubleshoot_image_pipeline(image_path, user_input="", location="Unknown", sentiment="neutral", chat_history=None):
    issues, captions = detect_image_issues(image_path)
    advice = generate_troubleshooting_advice(issues, captions, user_input, sentiment, chat_history)
    contractor_results = find_local_contractors(" ".join(issues or captions), location)

    # Save top 3 results for logs
    save_top3_output(captions[:3], issues[:3], contractor_results[:3])

    return {
        "captions": captions,
        "issues": issues,
        "advice": advice,
        "contractors": contractor_results
    }


In [ ]:
result = troubleshoot_image_pipeline(
    image_path="/content/floor damage.jpeg",
    user_input="",
    location="Brooklyn, NY",
    sentiment="0.125",
    chat_history=[""]
)

print("\n--- Captions ---")
print(result["captions"])
print("\n--- Issues ---")
print(result["issues"])
print("\n--- Advice ---")
print(result["advice"])
print("\n--- Local Contractors ---")
for c in result["contractors"]:
    print(c)


--- Captions ---
['a wooden floor in a living room', 'a room with wooden floors and a large window', 'a room with a wooden floor and a large window']

--- Issues ---
['wooden floor damage', 'Improper waterproofing layer', 'Cement flaking due to sulfate attack']

--- Advice ---
Based on the identified issues, here is a summary of the problems and potential solutions:

* Wooden floor damage: The wooden floor is likely suffering from water damage due to the improper waterproofing layer. This can cause warping, cracking, and rotting of the wood.
* Improper waterproofing layer: The waterproofing layer was not properly applied or has failed, allowing water to seep into the floor.
* Cement flaking due to sulfate attack: The sulfate attack is likely caused by the presence of sulfates in the concrete or mortar, which has reacted with the cement to cause flaking and degradation.

To address these issues, I recommend the following steps:
* Remove the damaged flooring material and inspect the sub

## **Agent 2 demo try** feature addition


# **Agent 2**

In [ ]:
import requests

GROQ_API_KEY = "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"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"
GROQ_MODEL = "llama3-8b-8192"  # Updated model

headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

def ask_tenancy_question(question, location=None):
    general_prompt = f"""
You are a legal tenancy assistant AI that helps tenants with their questions.
First, answer the following question in a general, globally applicable context:
"{question}"

Then, if location-specific laws or conditions might change the answer, ask the user for their location and tell them you'll provide more accurate help based on that.
Only if a location is provided, refine your answer further.
    """

    # Initial general response
    data = {
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": general_prompt}],
        "temperature": 0.7
    }

    response = requests.post(GROQ_URL, headers=headers, json=data)
    try:
        general_response = response.json()["choices"][0]["message"]["content"].strip()
        print("\n--- GENERAL TENANCY RESPONSE ---")
        print(general_response)
    except:
        print("\n[LLM ERROR] Could not generate response from Groq.")
        print("Response:\n", response.text)
        return

    if location:
        refined_prompt = f"""
The user previously asked: "{question}"
Now they have also provided their location: {location}
Please give a more detailed, location-specific answer based on tenancy laws or known conditions in that region.
        """
        data["messages"] = [{"role": "user", "content": refined_prompt}]
        refined_response = requests.post(GROQ_URL, headers=headers, json=data)

        try:
            location_specific = refined_response.json()["choices"][0]["message"]["content"].strip()
            print("\n--- LOCATION-SPECIFIC RESPONSE ---")
            print(location_specific)
        except:
            print("\n[LLM ERROR] Could not refine based on location.")
            print("Response:\n", refined_response.text)



In [ ]:
# Example usage
# ask_tenancy_question("Can my landlord enter the house without permission?")
# Optional with location
ask_tenancy_question("Can my landlord enter the house without permission?", location="Canada")


--- GENERAL TENANCY RESPONSE ---
A great question to start with!

In general, the answer is that a landlord typically needs permission to enter the rented property unless they have a valid reason to do so. This is because tenants have a reasonable expectation of privacy in their own home.

However, the laws regarding landlord access can vary depending on the jurisdiction. To provide more accurate guidance, I'd like to know your location. Please provide me with your country or region, and I'll do my best to provide more specific information on the laws and regulations that apply to you.

For example, in some jurisdictions, landlords may need to provide written notice to tenants before entering the property, while in others, they may need to obtain a court order. Additionally, some jurisdictions may have specific regulations or restrictions on when and how landlords can enter the property.

So, please let me know your location, and I'll be happy to provide more tailored advice.

--- LOC

# **Final Hosting**
Has all the basic functionality including chat hsitory

In [1]:
!pip install gradio whisper openai langchain faiss-cpu sentence-transformers wikipedia ipywidgets
!pip uninstall whisper -y
!pip install -U openai-whisper
!pip install -U langchain-community
!pip install -U langchain langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [5]:
# ---------------------- AGENT 1 SETUP ----------------------
import torch
from PIL import Image
import clip
import json
import requests
import torchvision.transforms as transforms
from transformers import BlipProcessor, BlipForConditionalGeneration

# ------------------- Model Setup -------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
blip_model.eval()

clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

# ------------------- Keys -------------------
SERPAPI_KEY = "345a9c709f5ac9c5828a76c34c4611d3b25f11f3414a43ba54097349ced2f85e"
GROQ_API_KEY = "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"
GROQ_MODEL = "llama3-8b-8192"

# ------------------- Utils -------------------
def save_top3_output(captions, issues, contractors, file_path="top3_results.json"):
    data = {
        "captions": captions,
        "issues": issues,
        "contractors": contractors
    }
    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)

def load_knowledge_base(path="issue_knowledge.txt"):
    with open(path, "r") as f:
        return [line.strip() for line in f.readlines() if line.strip()]

def detect_image_issues(image_path, knowledge_base_path="issue_knowledge.txt", top_k=3):
    image = Image.open(image_path).convert('RGB')

    # BLIP captioning
    inputs = blip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        caption_output = blip_model.generate(**inputs, max_length=100, num_beams=5, num_return_sequences=5)
    captions = [blip_processor.decode(out, skip_special_tokens=True) for out in caption_output][:top_k]

    # CLIP similarity to fallback if captions aren't strong
    issue_prompts = load_knowledge_base(knowledge_base_path)
    image_input_clip = clip_preprocess(image).unsqueeze(0).to(device)
    text_inputs = torch.cat([clip.tokenize(p) for p in issue_prompts]).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image_input_clip)
        text_features = clip_model.encode_text(text_inputs)

    similarity_scores = (image_features @ text_features.T).squeeze(0)
    scores_np = similarity_scores.cpu().numpy()
    max_score = scores_np.max()
    threshold = max(scores_np.mean() + scores_np.std(), 0.3)

    if max_score < threshold:
        return [], captions  # Fallback: use only captions

    top_indices = similarity_scores.topk(top_k).indices
    top_issues = [(issue_prompts[i], float(similarity_scores[i])) for i in top_indices]
    return [i[0] for i in top_issues], captions

def generate_troubleshooting_advice(issues, captions, user_input="", sentiment="neutral", chat_history=None):
    chat_history = chat_history or []

    prompt_sections = [
        "You are a repair assistant. Provide accurate, no-fluff guidance.",
        f"\n[ISSUES IDENTIFIED]\n{', '.join(issues) if issues else 'None'}",
        f"\n[CAPTIONS]\n{'; '.join(captions)}",
        f"\n[USER MESSAGE]\n{user_input if user_input else 'None'}",
        f"\n[SENTIMENT]\n{sentiment}",
        f"\n[CHAT HISTORY]\n" + "\n".join(chat_history[-3:]) if chat_history else ""
    ]

    full_prompt = "\n".join(prompt_sections) + "\n\nRespond in under 2 paragraphs. Use bullet points if needed. No emojis."

    try:
        response = requests.post(
            url="https://api.groq.com/openai/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {GROQ_API_KEY}",
                "Content-Type": "application/json"
            },
            json={
                "model": GROQ_MODEL,
                "messages": [{"role": "user", "content": full_prompt}],
                "temperature": 0.5
            }
        )
        data = response.json()
        if "choices" in data and data["choices"]:
            return data["choices"][0]["message"]["content"].strip()
        return "No response from model."

    except Exception as e:
        return f"LLM call failed: {e}"

def find_local_contractors(query, location, max_results=3):
    params = {
        "engine": "google",
        "q": f"{query} technician contact near {location}",
        "api_key": SERPAPI_KEY,
        "num": "10"
    }

    try:
        response = requests.get("https://serpapi.com/search", params=params)
        data = response.json()
        contractors = []
        helpful_websites = []

        for r in data.get("local_results", []):
            name = r.get("title") or r.get("name")
            phone = r.get("phone")
            address = r.get("address")
            if name:
                contractors.append(f"{name} - {phone or 'N/A'} - {address or 'N/A'}")
            if len(contractors) >= max_results:
                break

        if len(contractors) < max_results:
            for org in data.get("organic_results", []):
                link = org.get("link")
                title = org.get("title")
                if link and title:
                    helpful_websites.append(f"{title} - {link}")
                if len(helpful_websites) >= max_results:
                    break

        results = []
        if contractors:
            results.append("Contractors:")
            results.extend(contractors)
        if helpful_websites:
            results.append("Websites that might help:")
            results.extend(helpful_websites)

        return results

    except Exception as e:
        return [f"Contractor search failed: {e}"]

def troubleshoot_image_pipeline(image_path, user_input="", location="Unknown", sentiment="neutral", chat_history=None):
    issues, captions = detect_image_issues(image_path)
    advice = generate_troubleshooting_advice(issues, captions, user_input, sentiment, chat_history)
    contractor_results = find_local_contractors(" ".join(issues or captions), location)
    save_top3_output(captions[:3], issues[:3], contractor_results[:3])
    return {
        "captions": captions,
        "issues": issues,
        "advice": advice,
        "contractors": contractor_results
    }


# ---------------------- AGENT 2 ----------------------
def ask_tenancy_question(question, location=None):
    general_prompt = f"""
You are a legal tenancy assistant AI that helps tenants with their questions.
First, answer the following question in a general, globally applicable context:
\"{question}\"

Then, if location-specific laws or conditions might change the answer, ask the user for their location and tell them you'll provide more accurate help based on that.
Only if a location is provided, refine your answer further.
    """

    data = {
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": general_prompt}],
        "temperature": 0.7
    }

    response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers={
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }, json=data)

    try:
        general_response = response.json()["choices"][0]["message"]["content"].strip()
    except:
        return "[LLM ERROR] Could not generate response from Groq."

    if location:
        refined_prompt = f"""
The user previously asked: \"{question}\"
Now they have also provided their location: {location}
Please give a more detailed, location-specific answer based on tenancy laws or known conditions in that region.
        """
        data["messages"] = [{"role": "user", "content": refined_prompt}]
        refined_response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers={
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        }, json=data)

        try:
            location_specific = refined_response.json()["choices"][0]["message"]["content"].strip()
            return f"{general_response}\n\n--- LOCATION-SPECIFIC RESPONSE ---\n{location_specific}"
        except:
            return general_response + "\n\n[ERROR] Could not refine response based on location."
    else:
        return general_response


# ---------------------- GRADIO UI ----------------------
import gradio as gr
import tempfile
import os
from datetime import datetime

chat_log, caption_log, issue_log = [], [], []

def handle_input(user_input, image=None, audio=None, location="Unknown", sentiment="neutral"):
    if image is not None or audio is not None:
        if isinstance(image, str):  # Check if it's a file path (string)
            image_path = image
        else:  # Otherwise, handle as a file object
            image_path = image.name if image else None

        if audio is not None:
            # placeholder: convert audio to text (stub)
            user_input = "[Voice Message Transcribed] " + user_input

        results = troubleshoot_image_pipeline(image_path=image_path, user_input=user_input, location=location, sentiment=sentiment, chat_history=chat_log)
        chat_log.append(user_input)
        caption_log.extend(results['captions'])
        issue_log.extend(results['issues'])

        output = f"**Issues Detected:** {results['issues']}\n\n**Captions:** {results['captions']}\n\n**Advice:** {results['advice']}\n\n**Nearby Help:**\n" + "\n".join(results['contractors'])
        return output

    elif user_input:
        chat_log.append(user_input)
        response = ask_tenancy_question(user_input, location)
        return response

    return "Please provide a valid input."


iface = gr.Interface(
    fn=handle_input,
    inputs=[
        gr.Textbox(label="Type your question or description"),
        gr.Image(type="filepath", label="Upload an Image"),
        gr.Audio(type="filepath", label="Or Speak Your Query (Optional)"),
        gr.Textbox(label="Your Location (Optional)"),
        gr.Textbox(label="Describe how urgent or severe it feels (Optional sentiment)")
    ],
    outputs="markdown",
    title="🏡 Real Estate Multi-Agent Assistant",
    description="Upload an image or speak to route to Agent 1 (repair assistant). Type only to ask Agent 2 (tenancy legal assistant)."
)

iface.launch(debug=True)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]


  0%|                                               | 0.00/338M [00:00<?, ?iB/s]
  1%|▍                                     | 3.39M/338M [00:00<00:10, 34.4MiB/s]
  3%|█▎                                    | 11.2M/338M [00:00<00:05, 62.0MiB/s]
  6%|██▏                                   | 19.6M/338M [00:00<00:04, 73.9MiB/s]
  8%|███                                   | 26.7M/338M [00:00<00:04, 68.9MiB/s]
 10%|███▉                                  | 34.5M/338M [00:00<00:04, 72.0MiB/s]
 13%|█████                                 | 44.8M/338M [00:00<00:03, 82.4MiB/s]
 16%|█████▉                                | 52.7M/338M [00:00<00:03, 81.0MiB/s]
 19%|███████▏                              | 64.3M/338M [00:00<00:03, 93.4MiB/s]
 22%|████████▏                             | 73.3M/338M [00:00<00:03, 91.6MiB/s]
 24%|█████████▏                            | 82.1M/338M [00:03<00:21, 12.3MiB/s]
 29%|██████████▊                           | 96.5M/338M [00:03<00:12, 19.9MiB/s]
 33%|████████████▉         

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d360cbe2f18e631f3c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/PIL/Image.py", line 3471, in open
    fp.seek(0)
    ^^^^^^^
AttributeError: 'NoneType' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2136, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1662, in call_function
    prediction = await anyio

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d360cbe2f18e631f3c.gradio.live


# **Hosting Update 1**
Added Documentaion and mail

In [7]:
!pip install fpdf
!pip install gradio
!pip install Flask
!pip install Pillow  # for image processing
!pip install pydub   # for audio file handling

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=7e5365d7869667d6dc535b27c08dd6bb11e7ae246beba6af7b802b2f4f826723
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


## **Added Micro-Services**
Document generation + Email
Email concered authorities

In [9]:
import os
import json
import smtplib
from fpdf import FPDF
from email.message import EmailMessage
import gradio as gr
import requests
import traceback

# ---------------------- CONFIG ----------------------
USER_EMAIL = "harshchinchakar33@gmail.com"
SENDER_EMAIL = "harshchinchakar9921@gmail.com"
SENDER_PASSWORD = "gwcgznjlxjsujzwe"

GROQ_API_KEY = "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"
GROQ_MODEL = "llama3-8b-8192"

CHAT_HISTORY_PATH = "chat_history.txt"

# ---------------------- HELPERS ----------------------
def load_saved_data(path="top3_results.json"):
    try:
        with open(path, "r") as f:
            return json.load(f)
    except Exception as e:
        print(f"[ERROR] Failed to load JSON: {e}")
        traceback.print_exc()
        return {"captions": [], "issues": [], "contractors": []}

def read_chat_history(path=CHAT_HISTORY_PATH):
    try:
        with open(path, "r") as f:
            return f.read().splitlines()
    except Exception as e:
        print(f"[ERROR] Failed to read chat history: {e}")
        traceback.print_exc()
        return []

def generate_document_text(data, chat_log):
    try:
        prompt = f"""
        Create a structured technical report using the following:

        [CAPTIONS]: {data.get('captions', [])}
        [ISSUES]: {data.get('issues', [])}
        [CONTRACTORS]: {data.get('contractors', [])}
        [CHAT HISTORY]: {' | '.join(chat_log)}

        Format:
        - Title
        - Date & Time
        - Summary
        - Issues Identified
        - Captions
        - Recommendations
        - Local Contractor Suggestions
        - User Interactions Summary
        """

        response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers={
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        }, json={
            "model": GROQ_MODEL,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.5
        })

        return response.json()["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"[ERROR] Document generation failed: {e}")
        traceback.print_exc()
        return "Error generating document."

def create_pdf(content, filename="report.pdf"):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)
        for line in content.split('\n'):
            pdf.multi_cell(0, 10, line)
        pdf.output(filename)
        return filename
    except Exception as e:
        print(f"[ERROR] PDF creation failed: {e}")
        traceback.print_exc()
        return None

def send_email_with_attachment(to_email, subject, body, attachment_path):
    try:
        msg = EmailMessage()
        msg["Subject"] = subject
        msg["From"] = SENDER_EMAIL
        msg["To"] = to_email
        msg.set_content(body)

        with open(attachment_path, "rb") as f:
            msg.add_attachment(f.read(), maintype="application", subtype="pdf", filename=os.path.basename(attachment_path))

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(SENDER_EMAIL, SENDER_PASSWORD)
            smtp.send_message(msg)
    except Exception as e:
        print(f"[ERROR] Email sending failed: {e}")
        traceback.print_exc()

def generate_mail_to_authorities(doc_summary):
    try:
        prompt = f"""
        Compose a formal, structured email to a concerned authority describing a detected issue and requesting support.
        Summarize the issue from this:
        {doc_summary}
        Mention that a full report (PDF) has been generated and sent.
        """

        response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers={
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        }, json={
            "model": GROQ_MODEL,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.6
        })

        return response.json()["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"[ERROR] Authority email generation failed: {e}")
        traceback.print_exc()
        return "Error generating authority email."

# ---------------------- GRADIO ----------------------

document_display = gr.Textbox(label="Document Preview", lines=25)
authority_mail_display = gr.Textbox(label="Generated Mail to Authorities", lines=15)
authority_email_input = gr.Textbox(label="Authority Email")

def handle_document_and_mail():
    data = load_saved_data()
    chat_log = read_chat_history()

    content = generate_document_text(data, chat_log)
    pdf_path = create_pdf(content)

    if pdf_path:
        send_email_with_attachment(USER_EMAIL, "Your Issue Report", "Attached is your generated issue report.", pdf_path)

    return content, pdf_path

def handle_authority_mail(authority_email):
    data = load_saved_data()
    chat_log = read_chat_history()

    content = generate_document_text(data, chat_log)
    email_text = generate_mail_to_authorities(content)

    pdf_path = create_pdf(content)
    if pdf_path:
        send_email_with_attachment(authority_email, "Attention Required: Issue Report", email_text, pdf_path)

    return email_text

with gr.Blocks() as doc_ui:
    gr.Markdown("## 📄 Document Generation System")

    with gr.Row():
        doc_btn = gr.Button("📄 Generate Document & Mail to User")
        authority_btn = gr.Button("📬 Generate & Mail to Authorities")

    with gr.Row():
        document_display.render()
        authority_mail_display.render()
        authority_email_input.render()

    doc_btn.click(fn=handle_document_and_mail, outputs=[document_display, gr.File()])
    authority_btn.click(fn=handle_authority_mail, inputs=[authority_email_input], outputs=[authority_mail_display])

if __name__ == "__main__":
    doc_ui.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5f6343b2a9aab0455b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Final Micro-Services Dash**

In [10]:
import os
import json
import smtplib
from fpdf import FPDF
from email.message import EmailMessage
import gradio as gr
import requests
from datetime import datetime

# ---------------------- CONFIG ----------------------
USER_EMAIL = "harshchinchakar33@gmail.com"
SENDER_EMAIL = "harshchinchakar9921@gmail.com"
SENDER_PASSWORD = "gwcgznjlxjsujzwe"

GROQ_API_KEY = "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"
GROQ_MODEL = "llama3-8b-8192"

# ---------------------- HELPERS ----------------------
def load_saved_data(path="top3_results.json"):
    with open(path, "r") as f:
        return json.load(f)

def load_chat_history():
    with open("chat_history_agent1.txt", "r") as f1, open("chat_history_agent2.txt", "r") as f2:
        chat1 = f1.read().strip()
        chat2 = f2.read().strip()
    return chat1 + "\n" + chat2

def generate_document_text(data):
    timestamp = datetime.now().strftime("%B %d, %Y, %H:%M")
    prompt = f"""
You are a technical assistant generating field inspection reports. Use the data below:

CAPTIONS: {data['captions']}
ISSUES: {data['issues']}
CONTRACTORS: {data['contractors']}

Generate a plain text technical report using this structure:
Title: Field Inspection Report
Date & Time: {timestamp}
Issue Faced:
[List and explain the technical issues based on the data]

Recommendations:
[Provide recommendations to fix these issues]

Troubleshooting Methods:
[Provide practical steps to troubleshoot or investigate the issue]

Local Contractor Suggestions:
[List any available suggestions or web resources without links]

Use no special characters, no asterisks, and no chatbot-related content. Keep it professional and direct.
"""
    response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers={
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }, json={
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5
    })

    return response.json()["choices"][0]["message"]["content"].strip()

def create_pdf(content, filename="report.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)
    for line in content.split('\n'):
        pdf.multi_cell(0, 10, line)
    pdf.output(filename)
    return filename

def send_email_with_attachment(to_email, subject, body, attachment_path):
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = to_email
    msg.set_content(body + "\n\nRegards,\nHarsh Chinchakar")

    with open(attachment_path, "rb") as f:
        msg.add_attachment(f.read(), maintype="application", subtype="pdf", filename=os.path.basename(attachment_path))

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(SENDER_EMAIL, SENDER_PASSWORD)
        smtp.send_message(msg)

def generate_mail_to_authorities(doc_summary):
    prompt = f"""
Write a formal email to a concerned authority describing a technical issue and requesting support. Mention that a full technical PDF report is attached.

Context:
{doc_summary}

Conclude with:
Regards,
Harsh Chinchakar
"""
    response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers={
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }, json={
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.6
    })

    return response.json()["choices"][0]["message"]["content"].strip()

# ---------------------- GRADIO INTERFACE ----------------------

document_display = gr.Textbox(label="Document Preview", lines=25)
authority_mail_display = gr.Textbox(label="Generated Mail to Authorities", lines=15)
authority_email_input = gr.Textbox(label="Authority Email")

def handle_document_and_mail():
    try:
        data = load_saved_data()
        content = generate_document_text(data)
        pdf_path = create_pdf(content)
        send_email_with_attachment(USER_EMAIL, "Your Issue Report", "Attached is your generated technical report.", pdf_path)
        return content, pdf_path
    except Exception as e:
        return f"Error generating document: {str(e)}", None

def handle_authority_mail(authority_email):
    try:
        data = load_saved_data()
        content = generate_document_text(data)
        email_text = generate_mail_to_authorities(content)
        pdf_path = create_pdf(content)
        send_email_with_attachment(authority_email, "Attention Required: Technical Issue Report", email_text, pdf_path)
        return email_text
    except Exception as e:
        return f"Error generating authority mail: {str(e)}"

with gr.Blocks() as doc_ui:
    gr.Markdown("## 📄 Document Generation System")

    with gr.Row():
        doc_btn = gr.Button("📄 Generate Document & Mail to User")
        authority_btn = gr.Button("📬 Generate & Mail to Authorities")

    with gr.Row():
        document_display.render()
        authority_mail_display.render()
        authority_email_input.render()

    doc_btn.click(fn=handle_document_and_mail, outputs=[document_display, gr.File()])
    authority_btn.click(fn=handle_authority_mail, inputs=[authority_email_input], outputs=[authority_mail_display])

if __name__ == "__main__":
    doc_ui.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7d1f6e78907e0a3c47.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
